In [1]:
import sys


sys.path.append("/Users/aaron/phd/impact-of-contamination-on-taxonomy")

In [9]:
from workflow.external.gtdb_metadata import GtdbMetadataR207
from tqdm import tqdm
from collections import defaultdict

from matplotlib.ticker import ScalarFormatter, NullFormatter
import pandas as pd
import matplotlib.pyplot as plt
from workflow.gunc_helper.aggregate_max_css_level_merged import AggregateMaxCssLevelMerged

In [3]:
DF_META = GtdbMetadataR207().output().read_cached()
DF_META.head()

[12/02/2023 10:14:09] - 40.6MiB = /srv/home/uqamussi/projects/gunc-chimeras/output/external/gtdb_r207_metadata.h5 -> /tmp/gunc-cache/srv/home/uqamussi/projects/gunc-chimeras/output/external/gtdb_r207_metadata.h5


,ambiguous_bases,checkm_completeness,checkm_contamination,checkm_marker_count,checkm_marker_lineage,checkm_marker_set_count,checkm_strain_heterogeneity,coding_bases,coding_density,contig_count,...,gtdb_type_species_of_genus,ncbi_taxonomy,ncbi_taxonomy_unfiltered,domain,phylum,class,order,family,genus,species
gid,,,,,,,,,,,,,,,,,,,,,
GCA_000006155.2,1916,93.12,0.0,1171,g__Bacillus (UID902),324,0.0,4305660,80.178992,426,...,f,d__Bacteria;p__Firmicutes;c__Bacilli;o__Bacill...,d__Bacteria;x__Terrabacteria group;p__Firmicut...,d__Bacteria,p__Firmicutes,c__Bacilli,o__Bacillales,f__Bacillaceae_G,g__Bacillus_A,s__Bacillus_A anthracis
GCA_000007325.1,1,99.95,0.0,149,k__Bacteria (UID2329),89,0.0,1973459,90.754610,1,...,t,d__Bacteria;p__Fusobacteria;c__Fusobacteriia;o...,d__Bacteria;p__Fusobacteria;c__Fusobacteriia;o...,d__Bacteria,p__Fusobacteriota,c__Fusobacteriia,o__Fusobacteriales,f__Fusobacteriaceae,g__Fusobacterium,s__Fusobacterium nucleatum
GCA_000007385.1,0,99.82,0.0,481,c__Gammaproteobacteria (UID4202),276,0.0,4190634,84.805944,1,...,f,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,d__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Xanthomonadales,f__Xanthomonadaceae,g__Xanthomonas,s__Xanthomonas oryzae
GCA_000008085.1,0,73.13,0.0,149,k__Archaea (UID2),107,0.0,462902,94.299480,1,...,t,d__Archaea;p__Nanoarchaeota;c__;o__Nanoarchaea...,d__Archaea;x__DPANN group;p__Nanoarchaeota;o__...,d__Archaea,p__Nanoarchaeota,c__Nanoarchaeia,o__Nanoarchaeales,f__Nanoarchaeaceae,g__Nanoarchaeum,s__Nanoarchaeum equitans
GCA_000008885.1,0,100.00,0.0,134,k__Bacteria (UID2495),80,0.0,617456,87.831079,2,...,f,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,d__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Enterobacterales_A,f__Enterobacteriaceae_A,g__Wigglesworthia,s__Wigglesworthia glossinidia_A


In [71]:
from workflow.external.gtdb_metadata import GtdbMetadataR207Full

DF_META_FULL = GtdbMetadataR207Full().output().read_cached()
DF_META_FULL.head()

,ambiguous_bases,checkm_completeness,checkm_contamination,checkm_marker_count,checkm_marker_lineage,checkm_marker_set_count,checkm_strain_heterogeneity,coding_bases,coding_density,contig_count,...,trna_aa_count,trna_count,trna_selenocysteine_count,domain,phylum,class,order,family,genus,species
gid,,,,,,,,,,,,,,,,,,,,,
GCA_000006155.2,1916,93.12,0.0,1171,g__Bacillus (UID902),324,0.0,4305660,80.178992,426,...,16,31,0,d__Bacteria,p__Firmicutes,c__Bacilli,o__Bacillales,f__Bacillaceae_G,g__Bacillus_A,s__Bacillus_A anthracis
GCA_000007325.1,1,99.95,0.0,149,k__Bacteria (UID2329),89,0.0,1973459,90.754610,1,...,20,47,0,d__Bacteria,p__Fusobacteriota,c__Fusobacteriia,o__Fusobacteriales,f__Fusobacteriaceae,g__Fusobacterium,s__Fusobacterium nucleatum
GCA_000007385.1,0,99.82,0.0,481,c__Gammaproteobacteria (UID4202),276,0.0,4190634,84.805944,1,...,20,53,0,d__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Xanthomonadales,f__Xanthomonadaceae,g__Xanthomonas,s__Xanthomonas oryzae
GCA_000008085.1,0,73.13,0.0,149,k__Archaea (UID2),107,0.0,462902,94.299480,1,...,17,38,0,d__Archaea,p__Nanoarchaeota,c__Nanoarchaeia,o__Nanoarchaeales,f__Nanoarchaeaceae,g__Nanoarchaeum,s__Nanoarchaeum equitans
GCA_000008885.1,0,100.00,0.0,134,k__Bacteria (UID2495),80,0.0,617456,87.831079,2,...,20,34,0,d__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Enterobacterales_A,f__Enterobacteriaceae_A,g__Wigglesworthia,s__Wigglesworthia glossinidia_A


In [14]:
D_GID_TO_TAX = DF_META['gtdb_taxonomy'].to_dict()

In [43]:
def get_rank_to_gids():
    out = defaultdict(set)
    for gid, tax in D_GID_TO_TAX.items():
        for rank in tax.split(';'):
            out[rank].add(gid)
    return out

D_RANK_TO_GIDS = get_rank_to_gids()

In [4]:
DF_CSS = AggregateMaxCssLevelMerged().output().read_cached()
DF_CSS.head()

[12/02/2023 10:16:33] - 1.9MiB = /srv/home/uqamussi/projects/gunc-chimeras/output/gunc_helper/gunc_merged_max_css_level.h5 -> /tmp/gunc-cache/srv/home/uqamussi/projects/gunc-chimeras/output/gunc_helper/gunc_merged_max_css_level.h5


,n_genes_called,n_genes_mapped,n_contigs,taxonomic_level,proportion_genes_retained_in_major_clades,genes_retained_index,clade_separation_score,contamination_portion,n_effective_surplus_clades,mean_hit_identity,reference_representation_score,pass.GUNC,source
gid,,,,,,,,,,,,,
GCA_000143435.1,2189,2110,4,genus,0.89,0.86,0.55,0.03,0.06,0.95,0.82,False,gtdb
GCA_000153745.1,2202,2165,2,family,0.92,0.90,0.49,0.06,0.12,0.67,0.61,False,progenomes
GCA_000155005.1,6879,6313,84,genus,0.87,0.79,0.79,0.10,0.22,0.97,0.77,False,gtdb
GCA_000161815.1,4499,4335,8,genus,0.80,0.77,0.50,0.19,0.50,0.95,0.74,False,progenomes
GCA_000166735.2,3414,3247,793,genus,0.94,0.90,0.57,0.02,0.05,0.93,0.83,False,progenomes


In [8]:
BAC_GIDS = frozenset(DF_META[DF_META['domain'] == 'd__Bacteria'].index)
print(f'BAC_GIDS: {len(BAC_GIDS):,}')

FAIL_GIDS = frozenset(DF_CSS.index)
print(f'FAIL_GIDS: {len(FAIL_GIDS):,}')

FAIL_BAC_GIDS = frozenset(BAC_GIDS.intersection(FAIL_GIDS))
print(f'FAIL_BAC_GIDS: {len(FAIL_BAC_GIDS):,}')

BAC_GIDS: 311,480
FAIL_GIDS: 35,723
FAIL_BAC_GIDS: 34,846


In [13]:
def get_species_where_all_fail():
    out = set()
    out_gids = set()

    d_species_to_gids = defaultdict(set)

    for row in DF_META.itertuples():
        if row.domain == 'd__Bacteria':
            d_species_to_gids[row.species].add(row.Index)

    for species, gids in d_species_to_gids.items():
        if gids.issubset(FAIL_GIDS):
            out.add(species)
            for gid in gids:
                out_gids.add(gid)

    return frozenset(out), frozenset(out_gids)

S_SPECIES_ALL_FAIL, S_GIDS_IN_SP_ALL_FAIL = get_species_where_all_fail()
print(f'S_SPECIES_ALL_FAIL: {len(S_SPECIES_ALL_FAIL):,}')
print(f'S_GIDS_IN_SP_ALL_FAIL: {len(S_GIDS_IN_SP_ALL_FAIL):,}')

S_SPECIES_ALL_FAIL: 3,701
S_GIDS_IN_SP_ALL_FAIL: 4,431


In [15]:
def calculate_breakdown_of_failed_by_rank(gids):
    out = defaultdict(set)
    for gid in gids:
        tax = D_GID_TO_TAX[gid]
        for rank in tax.split(';'):
            out[rank[0]].add(rank)
    return out

In [20]:
DF_FAIL_BAC_BREAKDOWN = calculate_breakdown_of_failed_by_rank(FAIL_BAC_GIDS)
print(f'DF_FAIL_BAC_BREAKDOWN: {len(DF_FAIL_BAC_BREAKDOWN["p"]):,}')

DF_FAIL_BAC_BREAKDOWN: 101


In [19]:
DF_FAIL_BAC_BREAKDOWN_SP_FAIL = calculate_breakdown_of_failed_by_rank(S_GIDS_IN_SP_ALL_FAIL)
print(f'DF_FAIL_BAC_BREAKDOWN_SP_FAIL: {len(DF_FAIL_BAC_BREAKDOWN_SP_FAIL["p"]):,}')

DF_FAIL_BAC_BREAKDOWN_SP_FAIL: 80


In [38]:
from workflow.util.taxonomy import calculate_taxonomic_novelty

D_GID_TO_TAX_NOVELTY, _ = calculate_taxonomic_novelty(D_GID_TO_TAX)

In [75]:
from collections import Counter


def calculate_considering_novelty():
    out = defaultdict(set)



    # i) gtdb_is_sp_rep = no + ani_congruent = no (45 genomes)
    # ii) gtdb_is_sp_rep = no + ani_congruent = inconclusive (27 genomes)
    # iii) gtdb_is_sp_rep = yes + ani_congruent = no (120 genomes)
    gids_to_use = {'GCA_011523095.1', 'GCA_002291775.1', 'GCF_002835625.1', 'GCA_013213925.1', 'GCF_003495765.1', 'GCF_017151305.1', 'GCA_003487585.1', 'GCF_001812365.1', 'GCF_001815585.1', 'GCA_002703565.1', 'GCA_902619155.1', 'GCA_902539685.1', 'GCF_002035805.1', 'GCA_902568005.1', 'GCA_016939315.1', 'GCA_905214255.1', 'GCF_009494015.1', 'GCA_902363945.1', 'GCF_003462845.1', 'GCF_009493905.1', 'GCA_905214135.1', 'GCA_902573475.1', 'GCA_018370815.1', 'GCF_014284905.1', 'GCF_016806405.1', 'GCF_016806285.1', 'GCA_905208535.1', 'GCF_008269775.1', 'GCF_902845755.1', 'GCA_905215365.1', 'GCA_900756095.1', 'GCA_900548685.1', 'GCF_018499505.1', 'GCA_905193315.1', 'GCF_000964075.1', 'GCF_015667745.1', 'GCF_002913635.1', 'GCA_905212495.1', 'GCF_002912425.1', 'GCF_009649595.1', 'GCA_900549575.1', 'GCA_007096555.1', 'GCA_900760075.1', 'GCA_902388545.1', 'GCA_018383955.1', 'GCA_018239885.1', 'GCA_017394825.1', 'GCA_903931905.1', 'GCA_002728285.1', 'GCA_900765645.1', 'GCA_001695755.1', 'GCA_017515185.1', 'GCA_905200745.1', 'GCA_016707075.1', 'GCA_902528895.1', 'GCA_011523145.1', 'GCA_900761055.1', 'GCF_002929465.1', 'GCA_905214645.1', 'GCA_902593295.1', 'GCA_001509115.1', 'GCA_018056875.1', 'GCA_008668795.1', 'GCA_008668585.1', 'GCF_000698005.1', 'GCA_900765305.1', 'GCA_018363345.1', 'GCA_017465765.1', 'GCA_009493725.1', 'GCA_900759525.1', 'GCA_903846615.1', 'GCA_002731855.1', 'GCA_002712565.1', 'GCA_003519645.1', 'GCA_900759445.1', 'GCA_007713455.1', 'GCF_003935375.1', 'GCA_900751995.1', 'GCF_014764705.1', 'GCF_011045835.1', 'GCF_002989035.1', 'GCF_000935215.1', 'GCF_000146185.1', 'GCA_018363915.1', 'GCF_003064105.1', 'GCA_002720285.1', 'GCA_000761495.1', 'GCF_003052605.1', 'GCF_000421645.1', 'GCA_003520315.1', 'GCA_905214525.1', 'GCA_905215385.1', 'GCF_001544135.1', 'GCF_002216875.1', 'GCA_902542505.1', 'GCA_900757205.1', 'GCF_000438455.1', 'GCF_000181875.2', 'GCA_900552295.1', 'GCA_902617465.1', 'GCA_014871545.1', 'GCA_900555225.1', 'GCA_018375265.1', 'GCF_001553955.1', 'GCA_900548495.1', 'GCA_900754275.1', 'GCA_900761085.1', 'GCA_002690085.1', 'GCA_900555155.1', 'GCA_900757715.1', 'GCA_900757265.1', 'GCF_002156525.1', 'GCA_018067325.1', 'GCA_002703065.1', 'GCA_003987695.1', 'GCF_000188175.1', 'GCA_014647715.1', 'GCA_905213265.1', 'GCA_017470815.1', 'GCA_014871605.1', 'GCA_018240785.1', 'GCA_902615045.1', 'GCA_905478895.1', 'GCF_002192695.1', 'GCA_900755005.1', 'GCA_002279705.1', 'GCA_902546615.1', 'GCA_900761035.1', 'GCF_907163115.1', 'GCA_016302095.1', 'GCA_900754385.1', 'GCF_003697915.1', 'GCA_905197385.1', 'GCA_900552345.1', 'GCA_900551635.1', 'GCF_001720585.1', 'GCA_015232335.1', 'GCA_905214105.1', 'GCF_009469595.1', 'GCF_001975705.1', 'GCA_002683515.1', 'GCA_002728075.1', 'GCA_009391605.1', 'GCF_001571065.1', 'GCA_017857425.1', 'GCA_015264885.1', 'GCA_900752275.1', 'GCA_900555355.1', 'GCF_001713415.1', 'GCA_018365895.1', 'GCF_016860525.1', 'GCA_003535775.1', 'GCF_014205895.1', 'GCF_000761245.1', 'GCA_004124465.1', 'GCA_018372555.1', 'GCA_003233295.1', 'GCF_001552295.1', 'GCA_009936885.1', 'GCA_017525015.1', 'GCA_900557255.1', 'GCF_017311415.1', 'GCA_016777265.1', 'GCA_902560025.1', 'GCA_001543395.1', 'GCA_900752345.1', 'GCF_001870785.1', 'GCF_000008685.2', 'GCF_007559425.1', 'GCA_012964215.1', 'GCF_001945665.1', 'GCF_016899775.1', 'GCA_018371055.1', 'GCF_014205885.1', 'GCF_005144475.1', 'GCA_012961165.1', 'GCF_000222305.1', 'GCA_903920215.1', 'GCA_018894445.1', 'GCA_905216205.1', 'GCA_011620145.1', 'GCF_000170955.2', 'GCA_900756605.1', 'GCF_003284925.1', 'GCA_016882185.1', 'GCF_014201775.1', 'GCA_017940955.1', 'GCA_009392775.1', 'GCA_013372875.1', 'GCA_016777745.1', 'GCA_018433265.1', 'GCA_905215145.1'}

    # gids_to_use = S_GIDS_IN_SP_ALL_FAIL

    sources = list()

    for gid in tqdm(gids_to_use):
        tax = D_GID_TO_TAX[gid]

        for rank in tax.split(';'):
            gids_in_rank = D_RANK_TO_GIDS[rank]

            # if rank.startswith('s__SZUA'):
            #     print('')
            # S_GIDS_IN_SP_ALL_FAIL

            if gids_in_rank.intersection(FAIL_GIDS).intersection(gids_to_use) == gids_in_rank:
                out[rank[0]].add(rank)

                gid_src = DF_META_FULL.loc[gid]['ncbi_genome_category']
                sources.append(gid_src)
                if len(gids_in_rank) > 1:
                    print(rank)

    print(Counter(sources))

    return out

D_RANK_TO_UNQ_SP_REP_ALL_FAIL = calculate_considering_novelty()

 47%|████▋     | 90/192 [00:01<00:01, 88.65it/s]

s__Anaerococcus vaginalis_C


100%|██████████| 192/192 [00:02<00:00, 88.53it/s]

s__Anaerococcus vaginalis_C
Counter({'derived from metagenome': 42, 'none': 15, 'derived from environmental sample': 6})


In [70]:
df_meta_bac = DF_META[DF_META['domain'] == 'd__Bacteria']

for rank in ('domain', 'phylum', 'class', 'order', 'family', 'genus', 'species'):
    n_sp_fail = len(DF_FAIL_BAC_BREAKDOWN_SP_FAIL[rank[0]])
    n_fail = len(DF_FAIL_BAC_BREAKDOWN[rank[0]])
    n_rep_all_fail = len(D_RANK_TO_UNQ_SP_REP_ALL_FAIL[rank[0]])
    n_expected = len(df_meta_bac[rank].unique())
    print(f'Bacteria: {rank} (expected={n_expected:,})')
    # print(f'Species where at least one genome failed: {n_fail:,} ({n_fail/n_expected:.2%})')
    # print(f'Species where all genomes failed {n_sp_fail:,} ({n_sp_fail/n_expected:.2%})')
    print(f'Count where all lower ranks contain only failed: {n_rep_all_fail:,} ({n_rep_all_fail/n_expected:.2%})')
    print(', '.join(sorted(D_RANK_TO_UNQ_SP_REP_ALL_FAIL[rank[0]])))
    print('-' * 20)


Bacteria: domain (expected=1)
Count where all lower ranks contain only failed: 0 (0.00%)

--------------------
Bacteria: phylum (expected=169)
Count where all lower ranks contain only failed: 0 (0.00%)

--------------------
Bacteria: class (expected=428)
Count where all lower ranks contain only failed: 0 (0.00%)

--------------------
Bacteria: order (expected=1,460)
Count where all lower ranks contain only failed: 0 (0.00%)

--------------------
Bacteria: family (expected=3,650)
Count where all lower ranks contain only failed: 0 (0.00%)

--------------------
Bacteria: genus (expected=15,342)
Count where all lower ranks contain only failed: 0 (0.00%)

--------------------
Bacteria: species (expected=62,291)
Count where all lower ranks contain only failed: 62 (0.10%)
s__Allobacillus sp007559425, s__Anaerococcus vaginalis_C, s__Anaerovibrio sp016302095, s__Avimonas sp003519645, s__Bact-08 sp003520315, s__Borreliella bissettiae, s__Borreliella californiensis, s__Borreliella finlandensis, s

In [50]:
print(D_RANK_TO_UNQ_SP_REP_ALL_FAIL['c'])

{'c__DTXE01'}
